ETL: Carga de Datos

In [1]:
# Imports 
from pyspark.sql.types import IntegerType, StringType, DateType, LongType
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, length
from pyspark.sql.functions import when, col


In [2]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):        
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df
    
    def save_db(self, df, tabla):
        df.write.jdbc(
            url=self.url,
            table=tabla,
            mode='append',
            properties=self.properties
        )
        
def create_spark_session(path_jar_driver):    
    conf = SparkConf().set('spark.driver.extraClassPath', path_jar_driver)
    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    return sql_context.sparkSession    

def get_dataframe_from_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

In [3]:
# LLENAR CON EL USUARIO DE CADA UNO
db_user = 'Estudiante_52_202415'
db_psswd = 'Estudiante_202425423'


connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver"
}

source_db_string_connection = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional_ETL'
destination_db_string_connection = f'jdbc:mysql://157.253.236.120:8080/{db_user}'

# Driver de conexion
# LINUX
#path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'
# WINDOWS
path_jar_driver = 'C:\\Users\\Rodolfo\\OneDrive\\Maestria MIAD\\Semestre 1\\Ciclo 2\\Modelado de Datos y ETL\\Semana 2\\mysql-connector-j-9.0.0.jar'

In [4]:
spark = create_spark_session(path_jar_driver)

C:\Users\Rodolfo\anaconda3\envs\ETL\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [5]:
conn_orig = MySQLConnector(spark=spark, connection_properties=connection_properties, url=source_db_string_connection)
conn_dest = MySQLConnector(spark=spark, connection_properties=connection_properties, url=destination_db_string_connection)

## Proceso de ETL para una dimensión.

DESCRIPCION PROCESO

## Dimensión Condiciones de Pago


AJUSTAR EL BLOQUE DEL DISENO PARA EL NUEVO MODELO DADO EN CLASE


![Modelo Movimientos](./images/CondPago.png)

### Extraction

In [37]:
sql_Rs_CondicionDePago = 'RaSaTransaccional_ETL.FuenteCondicionesDePago_ETL'

df_Rs_CondicionDePago = conn_orig.get_dataframe(sql_Rs_CondicionDePago)
df_Rs_CondicionDePago = df_Rs_CondicionDePago \
    .withColumn('IdCondicionesDePago_T', col('IdCondicionesDePago_T').cast(LongType())) \
    .withColumn('Descripcion', col('Descripcion').cast(StringType())) \
    .withColumn('Tipo', col('Tipo').cast(StringType())) \
    .orderBy("IdCondicionesDePago_T")


In [38]:
df_Rs_CondicionDePago.show(40)

+---------------------+--------------------+-----------+
|IdCondicionesDePago_T|         Descripcion|       Tipo|
+---------------------+--------------------+-----------+
|                    9|           No Charge|Coseguridad|
|                   17|Copay per Day aft...|   Copagado|
|                   18|No Charge after d...|   Coseguro|
|                   27|Coinsurance after...|   Coseguro|
|                   34|      Not Applicable|   Copagado|
|                   34|      Not Applicable|   Copagado|
|                   36|      Not Applicable|   Coseguro|
|                   45|         Coinsurance|   Coseguro|
|                   45|         Coinsurance|   Coseguro|
|                   51|           No Charge|     Copago|
|                   68|Copay per Stay af...|     Copago|
|                   85|Copay per Day bef...|     Copago|
|                  102|Copay per Stay be...|     Copago|
|                  119|Copay per Day wit...|     Copago|
|                  136|Copay pe

## Transformation

### Realizar las transformaciones en la variable "Tipo" para que únicamente que valores Copago y Coseguro 

In [39]:
# Realizar las transformaciones en la columna "Tipo"
df_Rs_CondicionDePago = df_Rs_CondicionDePago.withColumn(
    "Tipo",
    when(col("Tipo") == "Copagado", "Copago")
    .when(col("Tipo") == "Coseguridad", "Coseguro")
    .otherwise(col("Tipo"))
)

# Mostrar los resultados para verificar las transformaciones
df_Rs_CondicionDePago.show(40)

+---------------------+--------------------+--------+
|IdCondicionesDePago_T|         Descripcion|    Tipo|
+---------------------+--------------------+--------+
|                    9|           No Charge|Coseguro|
|                   17|Copay per Day aft...|  Copago|
|                   18|No Charge after d...|Coseguro|
|                   27|Coinsurance after...|Coseguro|
|                   34|      Not Applicable|  Copago|
|                   34|      Not Applicable|  Copago|
|                   36|      Not Applicable|Coseguro|
|                   45|         Coinsurance|Coseguro|
|                   45|         Coinsurance|Coseguro|
|                   51|           No Charge|  Copago|
|                   68|Copay per Stay af...|  Copago|
|                   85|Copay per Day bef...|  Copago|
|                  102|Copay per Stay be...|  Copago|
|                  119|Copay per Day wit...|  Copago|
|                  136|Copay per Stay wi...|  Copago|
|                  136|Copay

### Incluir la variable de la base de datos IdCondicionDePago_DWH y organizar la estructura de la tabla

In [40]:
df_Rs_CondicionDePago = df_Rs_CondicionDePago.coalesce(1).withColumn('IdCondicionDePago_DWH', f.monotonically_increasing_id() + 1)
df_Rs_CondicionDePago = df_Rs_CondicionDePago.withColumnRenamed('IdCondicionesDePago_T', 'IdCondicionDePago_T')
df_Rs_CondicionDePago.show()

+-------------------+--------------------+--------+---------------------+
|IdCondicionDePago_T|         Descripcion|    Tipo|IdCondicionDePago_DWH|
+-------------------+--------------------+--------+---------------------+
|                  9|           No Charge|Coseguro|                    1|
|                 17|Copay per Day aft...|  Copago|                    2|
|                 18|No Charge after d...|Coseguro|                    3|
|                 27|Coinsurance after...|Coseguro|                    4|
|                 34|      Not Applicable|  Copago|                    5|
|                 34|      Not Applicable|  Copago|                    6|
|                 36|      Not Applicable|Coseguro|                    7|
|                 45|         Coinsurance|Coseguro|                    8|
|                 45|         Coinsurance|Coseguro|                    9|
|                 51|           No Charge|  Copago|                   10|
|                 68|Copay per Stay af

In [14]:
# Reorganizar las columnas en el orden de la BD
df_Rs_CondicionDePago = df_Rs_CondicionDePago.select(
    "IdCondicionDePago_DWH", "IdCondicionDePago_T", "Descripcion", "Tipo"
)

df_Rs_CondicionDePago.show()

+---------------------+-------------------+--------------------+--------+
|IdCondicionDePago_DWH|IdCondicionDePago_T|         Descripcion|    Tipo|
+---------------------+-------------------+--------------------+--------+
|                    1|                187|Copay with deduct...|  Copago|
|                    2|                204|       Copay per Day|  Copago|
|                    3|                 45|         Coinsurance|Coseguro|
|                    4|                 85|Copay per Day bef...|  Copago|
|                    5|                 18|No Charge after d...|Coseguro|
|                    6|                 27|Coinsurance after...|Coseguro|
|                    7|                238|               Copay|  Copago|
|                    8|                207|No Charge after d...|  Copago|
|                    9|                 68|Copay per Stay af...|  Copago|
|                   10|                153|Copay after deduc...|  Copago|
|                   11|               

## Load

In [19]:
# CARGUE
conn_dest.save_db(df_Rs_CondicionDePago, "Rs_CondicionDePago")  

## Dimensión Proveedor

### Extraction

In [49]:
sql_Rs_Proveedor = "(SELECT IdProveedor_T FROM FuentePlanesBeneficio_ETL) AS Rs_Proveedor"

df_Rs_Proveedor = conn_orig.get_dataframe(sql_Rs_Proveedor)
df_Rs_Proveedor.withColumn('IdProveedor_T', col('IdProveedor_T').cast(LongType()))

DataFrame[IdProveedor_T: bigint]

In [50]:
df_Rs_Proveedor.show()

+-------------+
|IdProveedor_T|
+-------------+
|        16842|
|        14002|
|        19722|
|        81413|
|        52697|
|        28162|
|        20129|
|        40572|
|        38166|
|        70893|
|        25268|
|        36096|
|        95185|
|        93078|
|        66252|
|        27248|
|        47840|
|        47840|
|        66252|
|        30751|
+-------------+
only showing top 20 rows



In [24]:
print((df_Rs_Proveedor.count(), len(df_Rs_Proveedor.columns)))

(70752, 1)


In [25]:
Vrunicos = df_Rs_Proveedor.distinct().count() 
print(f"Valores únicos en 'IdProveedor_T': {Vrunicos}")

Valores únicos en 'IdProveedor_T': 171


In [26]:
total_registros = df_Rs_Proveedor.count()
registros_sin_duplicados = df_Rs_Proveedor.dropDuplicates()
duplicados = total_registros - registros_sin_duplicados.count()
print(f"Cantidad de registros duplicados: {duplicados}")

Cantidad de registros duplicados: 70581


### Seleccionar los valores únicos de IdProveedor_T

In [54]:
# Seleccionar valores únicos de la columna IdProveedor_T
df_Rs_Proveedor = df_Rs_Proveedor.select("IdProveedor_T").distinct().orderBy("IdProveedor_T")

# Mostrar los resultados
df_Rs_Proveedor.show()

+-------------+
|IdProveedor_T|
+-------------+
|        10207|
|        11269|
|        11469|
|        11512|
|        12303|
|        12379|
|        12858|
|        14002|
|        14609|
|        15560|
|        15833|
|        16322|
|        16842|
|        16985|
|        18239|
|        18350|
|        19636|
|        19722|
|        20069|
|        20129|
+-------------+
only showing top 20 rows



In [28]:
print((df_Rs_Proveedor.count(), len(df_Rs_Proveedor.columns)))

(171, 1)


In [55]:
df_Rs_Proveedor = df_Rs_Proveedor.coalesce(1).withColumn('IdProveedor_DWH', f.monotonically_increasing_id() + 1)
df_Rs_Proveedor = df_Rs_Proveedor.select("IdProveedor_DWH", "IdProveedor_T")
df_Rs_Proveedor.show()

+---------------+-------------+
|IdProveedor_DWH|IdProveedor_T|
+---------------+-------------+
|              1|        10207|
|              2|        11269|
|              3|        11469|
|              4|        11512|
|              5|        12303|
|              6|        12379|
|              7|        12858|
|              8|        14002|
|              9|        14609|
|             10|        15560|
|             11|        15833|
|             12|        16322|
|             13|        16842|
|             14|        16985|
|             15|        18239|
|             16|        18350|
|             17|        19636|
|             18|        19722|
|             19|        20069|
|             20|        20129|
+---------------+-------------+
only showing top 20 rows



## Load

In [31]:
# CARGUE
conn_dest.save_db(df_Rs_Proveedor, "Rs_Proveedor")  

## Dimensión NivelesDeServicio

### Extraction

In [34]:
sql_Rs_NivelesDeServicio = 'RaSaTransaccional_ETL.NivelesDeServicio'

df_Rs_NivelesDeServicio = conn_orig.get_dataframe(sql_Rs_NivelesDeServicio)
df_Rs_NivelesDeServicio = df_Rs_NivelesDeServicio \
    .withColumn('IdNivelDeServicio_DWH', col('IdNivelDeServicio_DWH').cast(LongType())) \
    .withColumn('IdNivelDeServicio_T', col('IdNivelDeServicio_T').cast(LongType())) \
    .withColumn('Descripcion', col('Descripcion').cast(StringType()))

In [35]:
df_Rs_NivelesDeServicio.show()

+---------------------+-------------------+---------------+
|IdNivelDeServicio_DWH|IdNivelDeServicio_T|    Descripcion|
+---------------------+-------------------+---------------+
|                    1|                  1|        Nivel 1|
|                    2|                  2|        Nivel 2|
|                    3|                  3|Fuera de la red|
+---------------------+-------------------+---------------+



## Transformation

No existen Transformaciones

## Load

In [36]:
# CARGUE
conn_dest.save_db(df_Rs_NivelesDeServicio, "Rs_NivelesDeServicio")  